Goal is to see how aggregating time steps based on monthly transactions worked with an ltsm model> it does not appear to have pciked up significant trends due to a roc score of about 0.54 using 70% of the entire data set as training data. 

In [1]:
import torch.nn as nn
from torch.utils.data import Dataset
import torch
import numpy as np
import pandas as pd
import os
import sys
init_dir = os.getcwd()

In [2]:
df_consumers = pd.read_parquet(init_dir+"/cashflow-data/consumer_data.parquet")
df_transactions = pd.read_parquet(init_dir+"/cashflow-data/transactions.parquet")
merged_df = df_transactions.merge(df_consumers, on = "masked_consumer_id")
filtered_df = merged_df[merged_df["posted_date"] < merged_df["evaluation_date"]]
filtered_df["loan_type"] = filtered_df["masked_consumer_id"].str[2]
filtered_df["loan_type"] = filtered_df["loan_type"].astype(int)
dataframes_by_loans = {lt: pd.DataFrame() for lt in filtered_df.loan_type.unique()}
for key in dataframes_by_loans:
    dataframes_by_loans[key] = filtered_df[:][filtered_df.loan_type == key]
dataframes_by_loans[4]

C:\Users\manat\AppData\Local\Temp\ipykernel_10672\346573791.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["loan_type"] = filtered_df["masked_consumer_id"].str[2]
C:\Users\manat\AppData\Local\Temp\ipykernel_10672\346573791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["loan_type"] = filtered_df["loan_type"].astype(int)


,masked_consumer_id,posted_date,amount,category,masked_transaction_id,evaluation_date,FPF_TARGET,total_balance,loan_type
2204908,C04103940,2022-05-17,-6.98,16.0,C04T05229060,2023-01-20,0.0,1171.11,4
2204909,C04103940,2022-05-17,-4.99,16.0,C04T05229061,2023-01-20,0.0,1171.11,4
2204910,C04103940,2022-05-18,-14.82,18.0,C04T05229062,2023-01-20,0.0,1171.11,4
2204911,C04103940,2022-05-18,-25.00,14.0,C04T05229063,2023-01-20,0.0,1171.11,4
2204912,C04103940,2022-05-19,-7.76,18.0,C04T05229064,2023-01-20,0.0,1171.11,4
...,...,...,...,...,...,...,...,...,...
17738078,C04100784,2021-05-24,-84.96,22.0,C04T01072780,2022-03-31,1.0,2212.18,4
17738079,C04100784,2021-05-24,-25.00,15.0,C04T01072781,2022-03-31,1.0,2212.18,4
17738080,C04100784,2021-05-26,-1.00,1.0,C04T01072782,2022-03-31,1.0,2212.18,4
17738081,C04100784,2021-05-26,-5.00,1.0,C04T01072783,2022-03-31,1.0,2212.18,4


In [3]:
df4 = dataframes_by_loans[4].copy().dropna()
df4['posted_date'] = pd.to_datetime(df4['posted_date'])
df4['converted_date'] = (max(df4['posted_date']) - df4['posted_date']).dt.total_seconds()/3600
df4

,masked_consumer_id,posted_date,amount,category,masked_transaction_id,evaluation_date,FPF_TARGET,total_balance,loan_type,converted_date
2204908,C04103940,2022-05-17,-6.98,16.0,C04T05229060,2023-01-20,0.0,1171.11,4,7608.0
2204909,C04103940,2022-05-17,-4.99,16.0,C04T05229061,2023-01-20,0.0,1171.11,4,7608.0
2204910,C04103940,2022-05-18,-14.82,18.0,C04T05229062,2023-01-20,0.0,1171.11,4,7584.0
2204911,C04103940,2022-05-18,-25.00,14.0,C04T05229063,2023-01-20,0.0,1171.11,4,7584.0
2204912,C04103940,2022-05-19,-7.76,18.0,C04T05229064,2023-01-20,0.0,1171.11,4,7560.0
...,...,...,...,...,...,...,...,...,...,...
17738078,C04100784,2021-05-24,-84.96,22.0,C04T01072780,2022-03-31,1.0,2212.18,4,16200.0
17738079,C04100784,2021-05-24,-25.00,15.0,C04T01072781,2022-03-31,1.0,2212.18,4,16200.0
17738080,C04100784,2021-05-26,-1.00,1.0,C04T01072782,2022-03-31,1.0,2212.18,4,16152.0
17738081,C04100784,2021-05-26,-5.00,1.0,C04T01072783,2022-03-31,1.0,2212.18,4,16152.0


In [4]:
transaction_grouped = pd.DataFrame(columns = ['net_series', 'count_series', 'total_amount_series'] + [f"series_category_{i}" for i in range(36)])
ranges = 720 #30*24 = 720 = 30 day interval 
for id in df4['masked_consumer_id'].unique():
    categories = {f"series_category_{i}": [] for i in range(36)}
    cur = {'net_series': [], 'count_series': [], 'total_amount_series':[]}
    cur.update(categories)
    ser = df4[df4['masked_consumer_id'] == id]
    for x in range(int(ser['converted_date'].min()), int(ser['converted_date'].max()), int(ranges)):
        mask = ser[(ser['converted_date'] >= x) & (ser['converted_date'] < x + ranges)]
        cur['net_series'].append(mask['amount'].sum())
        cur['count_series'].append(mask['amount'].count())
        cur['total_amount_series'].append(mask['amount'].abs().sum())
        for i in range(36):
            cur[f"series_category_{i}"].append(mask[mask['category'] == i]['amount'].sum())
    if len(cur['net_series']) < 12: # 12*1 = 12 months recorded
        for x in cur.keys():
            cur[x] = [0]*(12 -  len(cur[x])) + cur[x][::-1]
    elif len(cur['net_series']) > 12:
        for x in cur.keys():
            cur[x] = cur[x][-12:][::-1]
    else:
        for x in cur.keys():
            cur[x] = cur[x][::-1]
    transaction_grouped.loc[id] = cur
transaction_grouped

,net_series,count_series,total_amount_series,series_category_0,series_category_1,series_category_2,series_category_3,series_category_4,series_category_5,series_category_6,...,series_category_26,series_category_27,series_category_28,series_category_29,series_category_30,series_category_31,series_category_32,series_category_33,series_category_34,series_category_35
C04103940,"[-1204.8799999999997, 580.9799999999994, 893.2...","[91, 101, 77, 92, 94, 76, 95, 102, 102, 98, 97...","[4745.999999999998, 6254.900000000001, 6316.96...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1694.5, 3043.0, 3127.68, 1604.96, 3070.0, 132...","[0.0, 0.0, 0.0, 340.0, 0.0, 0.0, 0.0, 0.0, 100...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[26.06, 107.94, 43.49, 10.98, 62.72, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 22.89, 134.62, 19.6, 0.0, 0.0,...",...,"[0.0, -16.63, -30.0, -98.31, -153.79, -115.2, ...","[0.0, 0.0, -21.08, -25.0, -92.87, 0.0, -40.34,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-19.91, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -6...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
C04103941,"[849.3400000000006, 2064.790000000001, -1138.7...","[90, 117, 112, 137, 120, 107, 122, 94, 113, 12...","[8294.32, 14785.329999999998, 11868.74, 9701.8...","[300.0, 150.0, 225.0, 375.56, 300.0, 0.0, 0.0,...","[-339.99, 1944.26, -805.55, -115.83, -601.4000...","[144.32, 1428.44, 294.66, 0.0, 343.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[163.28, 120.0, 129.12, 40.0, 40.0, 40.0, 41.0...","[3864.2300000000005, 3577.37, 3684.79, 4107.96...","[100.0, 100.0, 0.0, 22.4, 13.69, 0.0, 0.0, 0.0...",...,"[-411.01, -608.12, -294.61, -334.9099999999999...","[0.0, 0.0, -34.71, -14.67, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -43.68, -1..."
C04103942,"[455.4200000000005, 311.4500000000006, 2762.71...","[52, 78, 91, 131, 63, 82, 76, 68, 103, 83, 94,...","[12783.339999999998, 8877.289999999999, 13952....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[68.94999999999982, -79.99000000000001, -1148....","[0.0, 0.0, 3000.0, 0.0, 0.0, 0.0, 0.0, 100.0, ...","[4471.18, 4257.360000000001, 5209.190000000000...","[0.0, 0.0, 0.0, 0.0, 160.31, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[-385.82, -233.85999999999999, -106.69, -166.1...","[0.0, -52.27, -82.48, -22.53, -99.330000000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, -27.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
C04103943,"[263.4000000000001, 637.3000000000003, -501.83...","[39, 53, 41, 58, 52, 45, 48, 57, 49, 60, 48, 40]","[4091.5000000000005, 6034.999999999999, 4263.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, -18.77, -67.21, 0.0, -7.0, -27...","[370.0, 300.0, 0.0, 220.0, 0.0, 0.0, 352.0, 22...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1807.45, 2025.15, 1880.6100000000001, 2862.22.

In [5]:
df4_encoded = pd.get_dummies(df4, columns = ['category'], drop_first = False)
categories = {f"category_{i}.0": 'mean' for i in range(36)}
aggregate = {'amount': [ 'count', 'max', 'min', 'sum'], 'FPF_TARGET': ['mean'], 'total_balance': ['mean'], 'converted_date': ['max', 'min']}
aggregate.update(categories)
features = df4_encoded.groupby('masked_consumer_id').agg(aggregate)
features.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in features.columns]
merged_df = features.merge(transaction_grouped, left_index = True, right_index = True).reset_index(drop = True)
merged_df

,amount_count,amount_max,amount_min,amount_sum,FPF_TARGET_mean,total_balance_mean,converted_date_max,converted_date_min,category_0.0_mean,category_1.0_mean,...,series_category_26,series_category_27,series_category_28,series_category_29,series_category_30,series_category_31,series_category_32,series_category_33,series_category_34,series_category_35
0,2423,20800.00,-20800.00,6694.17,0.0,34412.26,23160.0,14568.0,0.005365,0.161370,...,"[0.0, 0.0, -1031.0, -922.0, -755.34, -914.0, -...","[-241.0, -327.0, -892.4, -472.84000000000003, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, -27.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-100.0, -400.0, -200.0, -300.0, -200.0, -400....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2171,6254.00,-6254.00,2972.58,0.0,5124.76,23088.0,14544.0,0.123445,0.249655,...,"[-50.0, -60.0, -60.0, -75.0, -105.0, -25.0, -1...","[-88.21000000000001, -4.8, -231.10000000000002...","[-127.66, -173.12, -53.79, -100.08, -53.79, -4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1477,5600.00,-4600.00,23.59,0.0,1959.64,23088.0,14520.0,0.004062,0.137441,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-62.16, -466.55, -635.23, -225.01999999999998...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, -3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -150.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,2985,11100.00,-16436.49,-7315.82,0.0,3176.35,23088.0,14568.0,0.142044,0.166499,...,"[-108.34, -58.35, -83.28999999999999, -246.57,...","[-191.76, -414.71, -1256.5800000000002, -621.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-189.84, -90.32, -168.07, -127.98, -135.97, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -75.0, -90..."
4,273,1087.49,-630.00,-2041.06,0.0,140.59,16320.0,14544.0,0.000000,0.183150,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, -19.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, -166.76, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,769,2000.00,-2944.86,-6255.18,0.0,1412.80,8592.0,0.0,0.039012,0.072822,...,"[-464.90000000000003, -341.39, -561.61, 0.0, -...","[-144.58, -124.16, -121.74, -62.72, 0.0, 0.0, ...","[0.0, 0.0, 0.0, -183.51, -132.72, -283.19, 0.0...","[-110.0, 0.0, 0.0, 0.0, -24.95, -24.95, -24.95...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,1147,4559

In [157]:
class TransactionDataset(Dataset):
    def __init__(self, merged_df):
        super().__init__()
        
        # Sequential features
        sequence_cols = ['net_series', 'count_series', 'total_amount_series'] + [f"series_category_{i}" for i in range(36)]
        self.sequences = []
        
        # Flatten sequences into shape (num_samples, time_steps, num_features)
        for idx, row in merged_df.iterrows():
            seq = []
            for t in range(len(row['net_series'])):  
                timestep_features = [
                    row['net_series'][t],
                    row['count_series'][t],
                    row['total_amount_series'][t]
                ] + [row[f'series_category_{i}'][t] for i in range(36)]
                seq.append(timestep_features)
            self.sequences.append(seq)
        
        self.sequences = torch.tensor(self.sequences, dtype=torch.float32)
        
        # Tabular  features 
        self.static_features = merged_df.drop(columns=sequence_cols + ['FPF_TARGET_mean']).values
        self.static_features = torch.tensor(self.static_features, dtype=torch.float32)

        self.targets = torch.tensor(merged_df['FPF_TARGET_mean'].values, dtype=torch.float32).unsqueeze(1)  

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.static_features[idx], self.targets[idx]


class TransactionLSTM(nn.Module):
    def __init__(self, seq_input_dim=39, static_input_dim=merged_df.shape[1]-39, hidden_dim=64, lstm_layers=2, output_dim=1):
        super().__init__()
        
        # LSTM for sequential data
        self.lstm = nn.LSTM(
                        input_size=seq_input_dim,
                        hidden_size=hidden_dim,
                        num_layers=lstm_layers,
                        batch_first=True,
                        bidirectional=True,
                        dropout=0.3,
                    )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim*2 + static_input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),  
        )

    def forward(self, sequence, static_features):
        lstm_out, _ = self.lstm(sequence)  
        lstm_last = lstm_out[:, -1, :]  
        combined = torch.cat([lstm_last, static_features], dim=1)
        out = self.fc(combined)
        return out



In [171]:
from sklearn.model_selection import train_test_split

# Train/test split
train_df, test_df = train_test_split(merged_df, test_size=0.3, random_state=42, stratify=merged_df['FPF_TARGET_mean'])

train_dataset = TransactionDataset(train_df)
test_dataset = TransactionDataset(test_df)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
model = TransactionLSTM(
    seq_input_dim=39, 
    static_input_dim=merged_df.shape[1]-39-1, 
    hidden_dim=128,
    lstm_layers=2,
    output_dim=1
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

criterion = nn.BCEWithLogitsLoss()

# Train the model
for epoch in range(20):
    model.train()
    for sequences, static_features, targets in train_loader:
        optimizer.zero_grad()
        
        outputs = model(sequences, static_features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}")
print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}")


Epoch 0: Train Loss = 3.1139
Epoch 5: Train Loss = 1.9515
Epoch 10: Train Loss = 0.0000
Epoch 15: Train Loss = 0.3224
Epoch 19: Train Loss = 1.7720


In [174]:
from sklearn.metrics import roc_auc_score

model.eval()
all_preds = []
all_targets = []    

with torch.no_grad():
    for sequences, static_features, targets in test_loader:
        outputs = model(sequences, static_features)  
        all_preds.append(outputs.cpu())
        all_targets.append(targets.cpu())

# Stack all
all_preds = (torch.cat(all_preds).squeeze().numpy() > 0.5).astype(int)
all_targets = (torch.cat(all_targets).squeeze().numpy() > 0.5).astype(int)


# Calculate ROC AUC
roc_auc = roc_auc_score(all_targets, all_preds)
print(f"Test ROC AUC: {roc_auc:.4f}")

Test ROC AUC: 0.5371


In [173]:
(all_preds == 1).mean()

0.0375